In [27]:
import os
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright, TimeoutError as PlaywrightTimeout
import time

In [28]:
SEASONS = list(range(1997, 2023))

In [29]:
P_DATA_DIR = "Playoff_DATA"
P_STANDINGS_DIR = os.path.join(P_DATA_DIR, "p_standings")
#PLAYOFFS_DIR = os.path.join(DATA_DIR, "playoffs")
P_SCORES_DIR = os.path.join(P_DATA_DIR, "p_scores")

git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

In [30]:
#get html function
async def get_html(url, selector, sleep=5, retries=3):
    html = None
    for i in range(1, retries+1):
        time.sleep(sleep * i)

        try:
            async with async_playwright() as p:
                browser = await p.firefox.launch()
                page = await browser.new_page()
                await page.goto(url)
                print(await page.title())
                html = await page.inner_html(selector)
        except PlaywrightTimeout:
            print(f"Timeout on {url}")
            continue
        else:
            break
    return html
    

In [31]:
# this function will scrape the playoff standings for each season
async def p_scrape_season(season):
    url = f"https://www.basketball-reference.com/playoffs/NBA_{season}_games.html"
    html = await get_html(url, "#info .button2.next")

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    href = [l["href"] for l in links]
    p_standings_pages = [f"https://www.basketball-reference.com{l}" for l in href ]

    for url in p_standings_pages:
        save_path = os.path.join(P_STANDINGS_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#schedule")
        with open(save_path, "w+") as f:
            f.write(html)

In [32]:
# this function will scrape the playoff scores for each season
for season in SEASONS:
    await p_scrape_season(season)

1997 NBA Playoffs Schedule and Results | Basketball-Reference.com
1998 NBA Playoffs Schedule and Results | Basketball-Reference.com
1999 NBA Playoffs Schedule and Results | Basketball-Reference.com
2000 NBA Playoffs Schedule and Results | Basketball-Reference.com
2001 NBA Playoffs Schedule and Results | Basketball-Reference.com
2002 NBA Playoffs Schedule and Results | Basketball-Reference.com
2003 NBA Playoffs Schedule and Results | Basketball-Reference.com
2004 NBA Playoffs Schedule and Results | Basketball-Reference.com
2005 NBA Playoffs Schedule and Results | Basketball-Reference.com
2006 NBA Playoffs Schedule and Results | Basketball-Reference.com
2007 NBA Playoffs Schedule and Results | Basketball-Reference.com
2008 NBA Playoffs Schedule and Results | Basketball-Reference.com
2009 NBA Playoffs Schedule and Results | Basketball-Reference.com
2010 NBA Playoffs Schedule and Results | Basketball-Reference.com
2011 NBA Playoffs Schedule and Results | Basketball-Reference.com
2012 NBA P

CancelledError: 

In [ ]:
standings_files = os.listdir(P_STANDINGS_DIR)

In [ ]:
standings_files

In [ ]:
async def scrape_game(standings_files):
    with open(standings_files, 'r') as f:
        html = f.read()

    soup = BeautifulSoup(html)
    links = soup.find_all("a")
    hrefs = [l.get("href") for l in links]
    box_scores = [l for l in hrefs if l and l.startswith("/boxscores")]
    box_scores = [f"https://basketball-reference.com{l}" for l in box_scores]

    for url in box_scores:
        save_path = os.path.join(P_SCORES_DIR, url.split("/")[-1])
        if os.path.exists(save_path):
            continue

        html = await get_html(url, "#content")
        with open(save_path, "w+") as f:
            f.write(html)

In [ ]:
standings_files = [s for s in standings_files if "html" in s]

In [ ]:
for f in standings_files:
    filepath = os.path.join(P_STANDINGS_DIR, f)
        
    await scrape_game(filepath)

In [ ]:
standings_files